In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.models import load_model

Using TensorFlow backend.


In [ ]:
def ModelTest(data_name,interval) :
    
    data = pd.read_csv("{}".format(data_name),header=None)
    test_cnt=len(data.iloc[:,0])/interval
    data.columns=['Math Expresion','Expression Name']
    data.loc[data["Expression Name"]=='distributor_expression','Expression Name']=0
    data.loc[data["Expression Name"]=='linear_expression','Expression Name']=1
    data.loc[data["Expression Name"]=='linear_equation','Expression Name']=2
    
    cnt=0
    for s in data.iloc[:,0]:
        data.iloc[cnt,0] = bytearray(s, 'utf-8')
        cnt+=1
        
    start = 0
    end = interval+1
    
    while(test_cnt>0):
        
        data_i = data.iloc[start:end,].copy()            
        X_train,X_test,y_train,y_test = train_test_split(data_i.iloc[:,0],data_i.iloc[:,1],test_size=0.2)
        X_train_seq = sequence.pad_sequences(X_train, maxlen=200)
        X_test_seq = sequence.pad_sequences(X_test, maxlen=200)
        Y_train_ctg = np_utils.to_categorical(y_train)
        Y_test_ctg = np_utils.to_categorical(y_test)
        model = Sequential()
        model.add(Embedding(len(X_train), 100))
        model.add(LSTM(100, activation='tanh'))
        model.add(Dense(3, activation='softmax'))
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        history = model.fit(X_train_seq, Y_train_ctg, batch_size=128, 
                        epochs=10,validation_data=(X_test_seq, Y_test_ctg))
        print("\n Test Accuracy: %.4f" % (model.evaluate(X_test_seq, Y_test_ctg)[1]))
        
        test_cnt-=1
        start +=interval
        end +=interval
        
        return model



def SaveModel(model,data_name) :
    model.save('{}.h5'.format(data_name))
    

def LoadModel(model_name) :
    model = load_model('{}.h5'.format(model_name))
    return model